In [1]:
from dataclasses import dataclass, field
from typing import List, Optional


# -----------------------------
# Environment: Bug ticket model
# -----------------------------

@dataclass
class BugTicket:
    id: int
    title: str
    description: str
    severity: Optional[str] = None        # e.g. "low", "medium", "high", "critical"
    category: Optional[str] = None        # e.g. "frontend", "backend", "infra", "other"
    assignee: Optional[str] = None        # e.g. owner's name
    status: str = "new"                   # e.g. "new", "triaged", "assigned", "notified"
    history: List[str] = field(default_factory=list)

    def add_history(self, event: str):
        self.history.append(event)


class BugDatabase:
    """Very simple in-memory 'database' of tickets."""
    def __init__(self):
        self.tickets: List[BugTicket] = []

    def add_ticket(self, ticket: BugTicket):
        self.tickets.append(ticket)

    def get_new_tickets(self) -> List[BugTicket]:
        return [t for t in self.tickets if t.status == "new"]


In [3]:
# -----------------------------
# Agents
# -----------------------------

class TriageAgent:
    """
    Reads ticket title & description, decides severity and category.
    Here it's rule-based; in a real system this could call an LLM.
    """

    def triage(self, ticket: BugTicket):
        text = (ticket.title + " " + ticket.description).lower()

        # --- Decide severity based on keywords ---
        if any(word in text for word in ["crash", "data loss", "security", "cannot login"]):
            severity = "critical"
        elif any(word in text for word in ["error", "fails", "not working", "slow"]):
            severity = "high"
        elif any(word in text for word in ["minor", "typo", "alignment"]):
            severity = "low"
        else:
            severity = "medium"

        # --- Decide category based on keywords ---
        if any(word in text for word in ["button", "ui", "screen", "browser", "layout"]):
            category = "frontend"
        elif any(word in text for word in ["api", "service", "database", "query", "logic"]):
            category = "backend"
        elif any(word in text for word in ["server", "deployment", "kubernetes", "network"]):
            category = "infra"
        else:
            category = "other"

        ticket.severity = severity
        ticket.category = category
        ticket.status = "triaged"
        ticket.add_history(f"TriageAgent: severity={severity}, category={category}")
        print(f"[TriageAgent] Ticket #{ticket.id}: severity={severity}, category={category}")


class AssignmentAgent:
    """
    Chooses an assignee based on category and severity.
    Again, rule-based for simplicity.
    """

    TEAM_MAP = {
        "frontend": "Alice",
        "backend": "Bob",
        "infra": "Carol",
        "other": "Dave",
    }

    SEVERITY_ESCALATION = {
        "critical": "Tech Lead",
        "high": "Senior Engineer",
    }

    def assign(self, ticket: BugTicket):
        # Base assignee by category
        base_assignee = self.TEAM_MAP.get(ticket.category or "other", "Dave")

        # Escalate based on severity
        if ticket.severity in self.SEVERITY_ESCALATION:
            assignee = f"{self.SEVERITY_ESCALATION[ticket.severity]} ({base_assignee})"
        else:
            assignee = base_assignee

        ticket.assignee = assignee
        ticket.status = "assigned"
        ticket.add_history(f"AssignmentAgent: assignee={assignee}")
        print(f"[AssignmentAgent] Ticket #{ticket.id}: assigned to {assignee}")


class NotifierAgent:
    """
    Pretends to notify the assignee (e.g., by email or chat).
    Here we just print to console.
    """

    def notify(self, ticket: BugTicket):
        message = (
            f"Hi {ticket.assignee},\n"
            f"You have a new {ticket.severity.upper()} bug assigned:\n"
            f"Title: {ticket.title}\n"
            f"Description: {ticket.description}\n"
        )
        # Here you might call Slack API / email API. We just print.
        print(f"[NotifierAgent] Sending notification for Ticket #{ticket.id}:\n{message}")
        ticket.status = "notified"
        ticket.add_history("NotifierAgent: notification sent")

In [4]:
# -----------------------------
# Orchestrator (Agentic system)
# -----------------------------

class BugWorkflowOrchestrator:
    """
    This is the 'agentic AI system' controller:
    - It looks for new tickets in the environment (BugDatabase)
    - Runs them through multiple agents in sequence
    - Maintains overall workflow
    """

    def __init__(self, db: BugDatabase,
                 triage_agent: TriageAgent,
                 assignment_agent: AssignmentAgent,
                 notifier_agent: NotifierAgent):
        self.db = db
        self.triage_agent = triage_agent
        self.assignment_agent = assignment_agent
        self.notifier_agent = notifier_agent

    def process_new_tickets(self):
        new_tickets = self.db.get_new_tickets()
        if not new_tickets:
            print("[Orchestrator] No new tickets to process.")
            return

        print(f"[Orchestrator] Processing {len(new_tickets)} new ticket(s)...")

        for ticket in new_tickets:
            # Step 1: triage
            self.triage_agent.triage(ticket)

            # Step 2: assign
            self.assignment_agent.assign(ticket)

            # Step 3: notify
            self.notifier_agent.notify(ticket)

            print(f"[Orchestrator] Ticket #{ticket.id} finished with status={ticket.status}")
            print(f"             History: {ticket.history}\n")




In [5]:
# -----------------------------
# Demo
# -----------------------------

def main():
    # Create environment + sample tickets
    db = BugDatabase()
    db.add_ticket(BugTicket(
        id=1,
        title="Login page crash on submit",
        description="The app crashes when user clicks the login button on Chrome."
    ))
    db.add_ticket(BugTicket(
        id=2,
        title="Slow response on search API",
        description="Search service is very slow when querying large datasets."
    ))
    db.add_ticket(BugTicket(
        id=3,
        title="Minor typo on settings page",
        description="The word 'account' is misspelled as 'acount' on the settings screen."
    ))

    # Create agents
    triage_agent = TriageAgent()
    assignment_agent = AssignmentAgent()
    notifier_agent = NotifierAgent()

    # Create orchestrator (agentic system)
    orchestrator = BugWorkflowOrchestrator(
        db=db,
        triage_agent=triage_agent,
        assignment_agent=assignment_agent,
        notifier_agent=notifier_agent,
    )

    # Run the workflow
    orchestrator.process_new_tickets()




In [6]:

if __name__ == "__main__":
    main()

[Orchestrator] Processing 3 new ticket(s)...
[TriageAgent] Ticket #1: severity=critical, category=frontend
[AssignmentAgent] Ticket #1: assigned to Tech Lead (Alice)
[NotifierAgent] Sending notification for Ticket #1:
Hi Tech Lead (Alice),
You have a new CRITICAL bug assigned:
Title: Login page crash on submit
Description: The app crashes when user clicks the login button on Chrome.

[Orchestrator] Ticket #1 finished with status=notified
             History: ['TriageAgent: severity=critical, category=frontend', 'AssignmentAgent: assignee=Tech Lead (Alice)', 'NotifierAgent: notification sent']

[TriageAgent] Ticket #2: severity=high, category=backend
[AssignmentAgent] Ticket #2: assigned to Senior Engineer (Bob)
[NotifierAgent] Sending notification for Ticket #2:
Hi Senior Engineer (Bob),
You have a new HIGH bug assigned:
Title: Slow response on search API
Description: Search service is very slow when querying large datasets.

[Orchestrator] Ticket #2 finished with status=notified
    